# Day 24: It Hangs in the Balance

[*Advent of Code 2015 day 24*](https://adventofcode.com/2015/day/24) and [*solution megathread*](https://www.reddit.com/3y1s7f)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2015/24/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2015%2F24%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


%load_ext nb_mypy
%nb_mypy On

Version 1.0.4


In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

%load_ext pycodestyle_magic
%pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [3]:
from IPython.display import HTML


HTML(downloaded['part1'])

## Comments

...

In [37]:
testdata_packages = frozenset([1, 2, 3, 4, 5, 7, 8, 9, 10, 11])
testdata_results = """Group 1;             Group 2; Group 3
11 9       (QE= 99); 10 8 2;  7 5 4 3 1
10 9 1     (QE= 90); 11 7 2;  8 5 4 3
10 8 2     (QE=160); 11 9;    7 5 4 3 1
10 7 3     (QE=210); 11 9;    8 5 4 2 1
10 5 4 1   (QE=200); 11 9;    8 7 3 2
10 5 3 2   (QE=300); 11 9;    8 7 4 1
10 4 3 2 1 (QE=240); 11 9;    8 7 5
9 8 3      (QE=216); 11 7 2;  10 5 4 1
9 7 4      (QE=252); 11 8 1;  10 5 3 2
9 5 4 2    (QE=360); 11 8 1;  10 7 3
8 7 5      (QE=280); 11 9;    10 4 3 2 1
8 5 4 3    (QE=480); 11 9;    10 7 2 1
7 5 4 3 1  (QE=420); 11 9;    10 8 2"""

inputdata = frozenset(int(p) for p in downloaded['input'].splitlines())

In [5]:
from typing import Iterable
from functools import reduce
import operator


def target_weight(packages: Iterable[int]) -> int:
    total_weight = sum(packages)
    if not total_weight % 3 == 0:
        raise ValueError('weight of packages is not divisible by three: ' +
                         f'{total_weight} ({packages})')
    return total_weight // 3


def calculate_QE(packages: Iterable[int]) -> int:
    return reduce(operator.mul, packages)

In [6]:
from typing import FrozenSet, \
    Tuple, \
    Iterator, \
    List

PackageGroup = FrozenSet[int]
PackageConfiguration = Tuple[PackageGroup, ...]


def take_group(permutation: Iterator[int],
               group_weight: int,
               max_len: int = 0) -> PackageGroup:
    output: List[int] = []
    w = 0
    try:
        while (w < group_weight and
               (max_len == 0 or len(output) < max_len)):
            output.append(next(permutation))
            w += output[-1]
    except StopIteration:
        raise ValueError('Exhausted permutation earlier than expected: ' +
                         f'{output=} {w=} ({group_weight=}')
    if w != group_weight:
        raise ValueError("Permutation did not fit into group: {output=}")
    return frozenset(output)

In [7]:
from typing import Set, Iterable

from itertools import permutations


def all_configurations(packages: List[int]) -> \
        Iterable[PackageConfiguration]:
    cache: Set[PackageConfiguration] = set()
    group_weight = target_weight(packages)
    for permutation in permutations(packages):
        permutation_iter = iter(permutation)
        try:
            configuration: PackageConfiguration = \
                tuple(take_group(permutation_iter, group_weight)
                      for _ in range(3))
        except ValueError:
            continue
        if configuration in cache:
            continue
        cache.add(configuration)
        yield configuration

In [8]:
def all_comfortable_configurations(packages: List[int]) -> \
        List[PackageConfiguration]:
    output: List[PackageConfiguration] = list()
    smallest_passenger_group = 0
    for configuration in all_configurations(packages):
        this_passenger_group = len(configuration[0])
        if smallest_passenger_group == 0 or \
                this_passenger_group < smallest_passenger_group:
            smallest_passenger_group = this_passenger_group
            output = [configuration]
        elif this_passenger_group > smallest_passenger_group:
            continue
        else:
            output.append(configuration)
    return output

In [9]:
def better_configurations(packages: List[int]) -> \
        Iterable[PackageConfiguration]:
    max_passenger_len: int = 0
    max_QE: int = 0
    cache: Set[PackageConfiguration] = set()
    configuration: PackageConfiguration = tuple()
    group_weight = target_weight(packages)
    for permutation in permutations(packages):
        display(f'{group_weight=} {max_passenger_len=} {max_QE=} ' +
                f'{permutation=}')
        permutation_iter = iter(permutation)
        try:
            passenger = take_group(permutation_iter,
                                   group_weight,
                                   max_passenger_len)
            # display(f'{group_weight=} {max_passenger_len=} {max_QE=} ' +
            #         f'{passenger=}')
            # Must pick the smallest sets for the
            # passenger compartment, regarless of QE
            this_passenger_len = len(passenger)
            this_QE = calculate_QE(passenger)
            if max_passenger_len == 0 or \
                    this_passenger_len < max_passenger_len:
                max_passenger_len = this_passenger_len
                max_QE = this_QE
            if max_QE != 0 and \
                    this_QE > max_QE:
                continue
            max_QE = this_QE
            left, right = (take_group(permutation_iter,
                                      group_weight)
                           for _ in range(2))
            configuration = (passenger, left, right)
        except ValueError:
            continue
        if configuration in cache:
            continue
        cache.add(configuration)
        yield configuration

<cell>9: error: Name "display" is not defined  [name-defined]


In [10]:
def best_configuration(packages: List[int]) -> \
        Tuple[int, PackageConfiguration]:
    for configuration in better_configurations(packages):
        QE = calculate_QE(configuration[0])
        # display(' '.join(str(set(g)) for g in configuration) +
        #         f' ({QE})')
        out_configuration = configuration
        out_QE = QE
    return out_QE, out_configuration

In [34]:
from itertools import combinations


def minimal_combinations(packages: PackageGroup) -> \
        FrozenSet[PackageGroup]:
    group_weight = target_weight(packages)
    for length in range(1, len(packages) - 1):
        output: FrozenSet[PackageGroup] = \
            frozenset(
                frozenset(pg)
                for pg in combinations(packages, length)
                if sum(pg) == group_weight)
        if output:
            return output
    raise ValueError('could not locate any combination for ' +
                     f'{group_weight} in {packages}')

In [33]:
def best_minimal_combination(packages: PackageGroup) -> \
        PackageGroup:
    return min(minimal_combinations(packages),
               key=lambda pc: calculate_QE(pc))

In [30]:
def remaining_configurations(packages: PackageGroup,
                             passenger_configuration: PackageGroup) -> \
        Iterable[PackageConfiguration]:
    cache: Set[PackageGroup] = set()
    group_weight = sum(passenger_configuration)
    remaining_packages = packages.difference(passenger_configuration)
    for left_length in range(1, len(remaining_packages)):
        for left in combinations(remaining_packages, left_length):
            if sum(left) != group_weight:
                continue
            left_fs = frozenset(left)
            if left_fs in cache:
                continue
            cache.add(left_fs)
            right_fs = remaining_packages.difference(left_fs)
            assert sum(right_fs) == group_weight
            yield passenger_configuration, left_fs, right_fs

In [39]:
def all_best_configurations(packages: PackageGroup) -> \
        Iterator[PackageConfiguration]:
    passenger = best_minimal_combination(packages)
    for configuration in remaining_configurations(packages,
                                                  passenger):
        yield configuration

In [40]:
display(list(str(conf) for conf in all_best_configurations(testdata_packages)))

<cell>1: error: Name "display" is not defined  [name-defined]


['(frozenset({9, 11}), frozenset({8, 2, 10}), frozenset({1, 3, 4, 5, 7}))',
 '(frozenset({9, 11}), frozenset({10, 3, 7}), frozenset({1, 2, 4, 5, 8}))',
 '(frozenset({9, 11}), frozenset({8, 5, 7}), frozenset({1, 2, 3, 4, 10}))',
 '(frozenset({9, 11}), frozenset({1, 2, 10, 7}), frozenset({8, 3, 4, 5}))',
 '(frozenset({9, 11}), frozenset({1, 10, 4, 5}), frozenset({8, 2, 3, 7}))',
 '(frozenset({9, 11}), frozenset({8, 1, 4, 7}), frozenset({10, 2, 3, 5}))',
 '(frozenset({9, 11}), frozenset({10, 2, 3, 5}), frozenset({8, 1, 4, 7}))',
 '(frozenset({9, 11}), frozenset({8, 2, 3, 7}), frozenset({1, 10, 4, 5}))',
 '(frozenset({9, 11}), frozenset({8, 3, 4, 5}), frozenset({1, 2, 10, 7}))',
 '(frozenset({9, 11}), frozenset({1, 2, 3, 4, 10}), frozenset({8, 5, 7}))',
 '(frozenset({9, 11}), frozenset({1, 2, 4, 5, 8}), frozenset({10, 3, 7}))',
 '(frozenset({9, 11}), frozenset({1, 3, 4, 5, 7}), frozenset({8, 2, 10}))']

In [41]:
display(list(str(conf) for conf in all_best_configurations(inputdata)))

<cell>1: error: Name "display" is not defined  [name-defined]


['(frozenset({1, 103, 107, 109, 79, 113}), frozenset({97, 2, 67, 101, 73, 83, 89}), frozenset({3, 5, 37, 71, 7, 41, 43, 13, 17, 19, 61, 53, 23, 59, 29, 31}))',
 '(frozenset({1, 103, 107, 109, 79, 113}), frozenset({97, 67, 3, 101, 71, 73, 17, 83}), frozenset({2, 5, 37, 7, 41, 43, 13, 19, 61, 53, 23, 89, 59, 29, 31}))',
 '(frozenset({1, 103, 107, 109, 79, 113}), frozenset({97, 67, 3, 71, 73, 83, 89, 29}), frozenset({2, 5, 37, 7, 101, 41, 43, 13, 17, 19, 53, 23, 59, 61, 31}))',
 '(frozenset({1, 103, 107, 109, 79, 113}), frozenset({67, 3, 101, 71, 73, 83, 53, 61}), frozenset({97, 2, 5, 37, 7, 41, 43, 13, 17, 19, 23, 89, 59, 29, 31}))',
 '(frozenset({1, 103, 107, 109, 79, 113}), frozenset({97, 67, 3, 71, 73, 53, 89, 59}), frozenset({2, 5, 37, 7, 101, 41, 43, 13, 17, 19, 83, 61, 23, 29, 31}))',
 '(frozenset({1, 103, 107, 109, 79, 113}), frozenset({97, 67, 3, 101, 71, 73, 41, 59}), frozenset({2, 5, 37, 7, 43, 13, 17, 19, 83, 53, 61, 23, 89, 29, 31}))',
 '(frozenset({1, 103, 107, 109, 79, 113}

In [42]:
calculate_QE({1, 103, 107, 109, 79, 113})

10723906903

In [11]:
# from IPython.display import display


# QE, configuration = best_configuration(testdata_packages)
# display(' '.join(str(set(g)) for g in configuration) +
#         f' ({QE})')

In [12]:
# QE, configuration = best_configuration(inputdata)
# display(' '.join(str(set(g)) for g in configuration) +
#         f' ({QE})')

In [13]:
HTML(downloaded['part1_footer'])

## Part Two

In [14]:
# HTML(downloaded['part2'])

In [15]:
# assert(my_part2_solution(testdata) == ...)

In [16]:
# my_part2_solution(inputdata)

In [17]:
# HTML(downloaded['part2_footer'])